In [1]:
from google.colab import drive
drive.mount('/content/drive/')

directory = '/content/drive/MyDrive/Colab Notebooks/data'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
import pickle

# write list to binary file
def write_list(a_list):
    # store list in binary file so 'wb' mode
    with open(r'/content/drive/MyDrive/Colab Notebooks/data/bad_draft_index.pickle', 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')
        
def read_list(filepath):
    # for reading also binary mode is important
    with open(filepath, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

In [1]:
import pandas as pd
import numpy as np
import gzip
import csv
import json
import dask.dataframe as dd
import regex as re
import logging
from sklearn.model_selection import train_test_split

In [2]:
import os
os.getcwd()

'c:\\Users\\aguay\\Documents\\GitHub\\rsaxe FULL\\mtg\\mtg'

In [5]:
bad_index = read_list(r'data\bad_draft_index.pickle')

In [6]:
COLUMN_REGEXES = {
    re.compile(r'user_game_win_rate_bucket'): 'float16',
    re.compile(r'user_n_games_bucket'): 'int8',
    re.compile(r'draft_id'): 'str',
    re.compile(r'draft_time'): 'str',
    re.compile(r'expansion'): 'str',
    re.compile(r'event_type'): 'str',
    re.compile(r'event_match_wins'): 'int8',
    re.compile(r'event_match_losses'): 'int8',
    re.compile(r'pack_number'): 'int8',
    re.compile(r'pick_number'): 'int8',
    re.compile(r'pick'): 'str',
    re.compile(r'pick_maindeck_rate'): 'float16',
    re.compile(r'pick_sideboard_in_rate'): 'float16',

    re.compile(r'pool_.*'): 'int8',
    re.compile(r'pack_card_.*'): 'int8',
}

def load_data(filename):
    col_names = pd.read_csv(filename, nrows=0).columns
    data_types = {}
    for c in col_names:
        for (r, t) in COLUMN_REGEXES.items():
            if r.match(c):
                data_types[c] = t
    #skipcols= ['draft_time',
    #           'event_type',
    #           'expansion',
    #           'event_match_wins',
    #           'event_match_losses',
    #           'user_n_games_bucket',
    #           'user_game_win_rate_bucket',
    #           'pick_maindeck_rate',
    #           'pick_sideboard_in_rate',
    #           'draft_id',
    #           'pick_number',
    #           'pack_number',
    #           'rank'
    #            ]
    df = pd.read_csv(
        filename,
        dtype=data_types,
        #nrows=100,
        skiprows=bad_index,
        #chunksize=420000,
        #usecols = lambda x: x not in skipcols
        #usecols = ['rank', 'pack_number', 'pick_number']      
    )
    #df['rank'] = df['rank'].fillna('bronze')
    return df

In [ ]:
#loading dataset with pandas
df = load_data("/content/drive/MyDrive/Colab Notebooks/data/draft_data_public.DMU.PremierDraft.csv")
#

In [ ]:
df.head()

In [ ]:
# loading dataset with dask
#df = dd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/draft_data_public.DMU.PremierDraft.csv", dtype={'rank': 'string'})

In [ ]:
df['pick'] = df['pick'].astype('string')

In [ ]:
df['rank'] = df['rank'].fillna('bronze')

In [ ]:
df['rank'] = df['rank'].astype('string')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.loc[df['draft_id']=='614cf9e25fb24fd0b047f9887bed40c4']
#614cf9e25fb24fd0b047f9887bed40c4

,draft_id,pack_number,pick_number


Train/Val/Test Splits

In [10]:
train_chunks = []
val_chunks = []
test_chunks = []

# Create a dictionary to track the draft_id and its corresponding set
draft_id_sets = {}

for chunk in load_data("data\DMU_draft_data.csv"):
    # Get unique draft_id values in the current chunk
    unique_draft_ids = chunk['draft_id'].unique()

    # Split the unique draft_id values into train, val, and test sets
    train_ids, val_test_ids = train_test_split(unique_draft_ids, test_size=0.2, random_state=42)
    val_ids, test_ids = train_test_split(val_test_ids, test_size=0.5, random_state=42)

    # Filter the chunk based on the draft_id sets
    train_chunk = chunk[chunk['draft_id'].isin(train_ids)]
    val_chunk = chunk[chunk['draft_id'].isin(val_ids)]
    test_chunk = chunk[chunk['draft_id'].isin(test_ids)]

    # Append the splits to their respective lists
    train_chunks.append(train_chunk)
    val_chunks.append(val_chunk)
    test_chunks.append(test_chunk)

# Concatenate the chunks into dataframes if needed
train_df = pd.concat(train_chunks)
val_df = pd.concat(val_chunks)
test_df = pd.concat(test_chunks)

In [ ]:
train_chunks = []
val_chunks = []
test_chunks = []

for chunk in load_data("/content/drive/MyDrive/Colab Notebooks/data/draft_data_public.DMU.PremierDraft.csv"):
    # Split the chunk into train, validation, and test sets
    train_chunk, val_test_chunk = train_test_split(chunk, test_size=0.2, random_state=42)
    val_chunk, test_chunk = train_test_split(val_test_chunk, test_size=0.5, random_state=42)

    # Append the splits to their respective lists
    train_chunks.append(train_chunk)
    val_chunks.append(val_chunk)
    test_chunks.append(test_chunk)

In [ ]:
# Concatenate the split chunks into single DataFrames
train_df = pd.concat(train_chunks, axis=0)
train_df.to_csv('data/finaltrain.csv', index=False)




In [11]:
# Write the split DataFrames to separate files
val_df = pd.concat(val_chunks, axis=0)
test_df = pd.concat(test_chunks, axis=0)
val_df.to_csv('data/finalval.csv', index=False)
test_df.to_csv('data/finaltest.csv', index=False)

In [ ]:
# Split train dataset into smaller files for easier processsing
# https://www.quicktable.io/blog/how-to-split-a-csv-file-into-multiple-files-cld2hn8hp113571mldlj9ua8z2
import csv 
# Open the input file 
with open("data.csv", "r") as input_file: 
    # Create a CSV reader object 
    reader = csv.reader(input_file) 
    # Initialize a counter variable 
    count = 0 
    # Iterate over the rows in the input file 
    for row in reader: 
        # If the counter is equal to 0, create a new output file 
        if count == 0: 
            # Increment the counter 
            count += 1 
            # Create a new output file with the name "data_part_1.csv" 
            with open("data_part_1.csv", "w") as output_file: 
                # Create a CSV writer object 
                writer = csv.writer(output_file) 
                # Write the first row to the output file 
                writer.writerow(row) 
        # If the counter is not equal to 0, write the row to the existing output file 
        else: 
            # Increment the counter 
            count += 1 
            # Write the row to the existing output file 
            writer.writerow(row) 
        # If the counter is equal to 1000, reset the counter 
        if count == 1000: 
            count = 0 

Dealing with Bad Drafts (those that have fewer than 42 picks)

In [ ]:
drafts = df.iloc[:,0]
#drafts = drafts.compute()
drafts = drafts.astype("string")
#
# 180251 unique drafts
# 7519894 rows

In [ ]:
drafts.info()
#count                              7519894
#unique                              180251
#top       562c07bcca0342d79365485503d153e4
#freq                                    42
#Name: draft_id, dtype: object

<class 'pandas.core.series.Series'>
RangeIndex: 7519894 entries, 0 to 7519893
Series name: draft_id
Non-Null Count    Dtype 
--------------    ----- 
7519894 non-null  string
dtypes: string(1)
memory usage: 57.4 MB


In [ ]:
bad = drafts.value_counts()<42

In [ ]:
bad = bad.to_frame()

In [ ]:
bad = bad.rename({'draft_id': 'bad_draft'}, axis='columns')

In [ ]:
bad.head()

,bad_draft
562c07bcca0342d79365485503d153e4,False
291f1fdd7c0c411ba58fd2208bb90086,False
c1e8e3025db74878a7526ac2491ec0e9,False
b2b00654381b443895ef9d0681438c33,False
1b73d69c2f5645d5a7531984c3bddbcf,False


In [ ]:
drafts = drafts.to_frame()

In [ ]:
drafts.head()

,draft_id,bad_draft
0,562c07bcca0342d79365485503d153e4,False
1,562c07bcca0342d79365485503d153e4,False
2,562c07bcca0342d79365485503d153e4,False
3,562c07bcca0342d79365485503d153e4,False
4,562c07bcca0342d79365485503d153e4,False


In [ ]:
drafts = drafts.join(bad, on="draft_id")

In [ ]:
bad_indexlist = list(np.where(drafts["bad_draft"] == True)[0])

In [ ]:
write_list(bad_indexlist)

Done writing list into a binary file


In [ ]:
bad_index = read_list('/content/drive/MyDrive/Colab Notebooks/data/bad_draft_index.pickle')

In [ ]:
print(len(bad_indexlist))
type(bad_indexlist)

73336


list

In [ ]:
bad_indexlist = list(map(lambda x:x+1, bad_indexlist))

In [ ]:
bad_indexlist

In [ ]:
drafts.iloc[1417]

draft_id     614cf9e25fb24fd0b047f9887bed40c4
bad_draft                                True
Name: 1417, dtype: object

In [13]:
sum(drafts.value_counts(ascending=True)<42)
# 2952 bad drafts to remove
# 73336 rows removed, about 25 rows per bad draft
# leaves 177299 good drafts with 42 picks each
# 7446558 total picks (rows)

0

Exploring Ranks Distribution

In [ ]:
ranks = df.iloc[:,1]
ranks = ranks.compute()
# 6 mins

In [ ]:
ranks.describe()

Dask Series Structure:
npartitions=1
    object
       ...
Name: rank, dtype: object
Dask Name: describe-nonnumeric, 22 graph layers

In [ ]:
ranks.isna().sum()

499954

In [ ]:
ranks.value_counts()

platinum    2912155
gold        1470430
silver      1102957
diamond     1032125
mythic       502273
Name: rank, dtype: int64

Exploring Distributions of Rank, Pick & Pack Number in Data Files

In [ ]:
#distributions of rank in original dataset

#platinum    2912155
#proportion  0.38726

#gold        1470430
#proportion  0.19554

#silver      1102957
#proportion  0.14667

#diamond     1032125
#proportion  0.13725

#mythic      502273
#proportion  0.06679

#bronze      499954
#proportion  0.06648

#total       7519894
d = {'platinum': 0.38726, 'gold': 0.19554, 'silver': 0.14667
     , 'diamond': 0.13725, 'mythic': 0.06679, 'bronze': 0.06648}

rankproportions = pd.Series(data=d, index=['platinum', 'gold', 'silver'
                  , 'diamond', 'mythic', 'bronze'])


In [ ]:
rankproportions

platinum    0.38726
gold        0.19554
silver      0.14667
diamond     0.13725
mythic      0.06679
bronze      0.06648
dtype: float64

In [ ]:
#loading dataset with pandas, change last part to val.csv, test.csv or train.csv
df = load_data("/content/drive/MyDrive/Colab Notebooks/data/test.csv")

In [ ]:
df['pack_number'] = df['pack_number'].astype('string')
df['pick_number'] = df['pick_number'].astype('string')

In [ ]:
dfrankproportions = df['rank'].value_counts()/sum(df['rank'].value_counts())
dfrankproportions-rankproportions
#val: difference caps at 0.001042 from expected proportion
#test: difference caps at 0.001008 from expected proportion
#train: difference caps at 0.000920 from expected proportion

platinum    0.000430
gold        0.000202
silver      0.000105
diamond     0.000062
mythic      0.000131
bronze     -0.000920
dtype: float64

In [ ]:
#distributions of pick & pack number in original dataset
#after eliminating bad drafts, we should be left with drafts that only feature 42 picks
#42 picks, 14 picks per pack, 3 packs
#so there should be an equal number of each pick/pack combination in each train/val/test sets
#proportion of each should be 1/42 = 0.023809523809

In [ ]:
df['packpicknumber'] = df['pack_number']+df['pick_number']

In [ ]:
ppproportions = df['packpicknumber'].value_counts()/sum(df['packpicknumber'].value_counts())
ppproportions-0.023809523809
#val: difference caps at 0.000829 from expected proportion
#test: difference caps at 0.000509 from expected proportion
#train: difference caps at 0.000045 from expected proportion

211    0.000509
013    0.000492
27     0.000492
01     0.000488
19     0.000482
11     0.000481
13      0.00048
113    0.000478
09     0.000478
23     0.000478
05     0.000476
15     0.000476
21     0.000473
17     0.000473
111    0.000471
011    0.000471
29     0.000471
213    0.000463
07     0.000462
25     0.000461
03     0.000449
012    -0.00045
08    -0.000454
04    -0.000455
00    -0.000463
24    -0.000463
12    -0.000463
20    -0.000467
28    -0.000471
210   -0.000474
16    -0.000477
26    -0.000478
14    -0.000481
110   -0.000482
10    -0.000483
18    -0.000487
112   -0.000487
212   -0.000487
22    -0.000489
02    -0.000491
06    -0.000498
010   -0.000502
Name: packpicknumber, dtype: Float64